In [1]:
import os
import random
import math

import pandas as pd
import numpy as np

In [ ]:
DATA_PATH = '/home/leelin/Downloads/2021_4_data'

config = {'data_path':'/home/leelin/Downloads/2021_4_data',
		  'output_path': '/tmp'
		 }

In [45]:
class NormalizedData(object):
	def __init__(self, config):
		'''
			Args:
				@param config: dictionary of the config
		'''
		self.en_train_data_path = os.path.join(
		    config['data_path'], 'train/train_en.tsv')
		self.tr_train_data_path = os.path.join(
		    config['data_path'], 'train/train_tr.tsv')
		self.predict_path = os.path.join(config['data_path'], 'to_predict.csv')
		self.en_doc_info_path = os.path.join(
		    config['data_path'], 'doc_info/en_list_result/')
		self.tr_doc_info_path = os.path.join(
		    config['data_path'], 'doc_info/tr_list_result/')

		self.output_path = config['output_path']

		self.part_range = 30000  # num of file num in doc_info 'part-xxxx'

	# def file_line_iter(self, file_path, is_doc_info=False):
	# 	'''
	# 		Args:
	# 			@param file_path: dir path needed while is_doc_info is True, otherwise
	# 				just normal file path
	# 	'''
	# 	if is_doc_info == True:

	# 	else:

	def gen_url2docidx(self, output_dir=None, max_line=100000):
		'''
			Args:
				@param output_name: if setted, will save DataFrame to target dir
				@param max_line: max number of rows per DataFrame
			return a DataFrame with columns = ['url','file_name','line_idx']
		'''

	def normalize_data(self, doc_info=False, predict_file=False, train_dir=False):
		'''
			Args:

		'''
		# ! 此处应该用如何替代
		replecment_pool = [
						['\x01', ' '],
						['<br>', ' ']]

		def inplace_modify(path, is_tr=False):
			# print("Normalize file: ", path)
			with open(path, 'r', encoding='utf8') as fr:
				lines = fr.readlines()
			for id in range(len(lines)):
				lines[id] = lines[id].lower()
				for key in replecment_pool:
					lines[id] = lines[id].replace(key[0], key[1])
				if is_tr:
					lines[id] = lines[id].replace('ı', 'i') \
                               .replace('ğ', 'g') \
                               .replace('ç', 'c') \
                               .replace('ö', 'o') \
                               .replace('ş', 's') \
                               .replace('ü', 'u')
				# print(lines[id])
			with open(path, 'w', encoding='utf8') as fr:
				fr.writelines(lines)

		if doc_info:
			for part_num in range(self.part_range):
				inplace_modify(self.en_doc_info_path+'part-%05d' % part_num)
				inplace_modify(self.tr_doc_info_path+'part-%05d' % part_num, is_tr=True)
		if predict_file:
			inplace_modify(self.predict_path)
		if train_dir:
			inplace_modify(self.en_train_data_path)
			inplace_modify(self.tr_train_data_path, is_tr=True)

	def train_test_split(*arrays, test_size=0.33, eval_size=None, shuffle=True, group_size=1):
		'''
			Splite the iterable data on index shape[0]

			Args:
				@param *arrays: pandans DataFrame or numpy array
				@param test_size: test data set size, float
				@param shuffle: use shuffle or not
				@param group_size: mulitple lines as a group (DIGIX-predict be top100)
			Return:
				(arrays[i]_train, arrays[i]_test, ...)
		'''
		# TODO: add gen_eval data
		result = []

		for array in arrays:
			idx = [(i, i+group_size) for i in range(0, len(array), group_size)]
			if shuffle:
				random.shuffle(idx)
			splite_idx = math.floor(len(idx) * test_size)

			if isinstance(array, pd.DataFrame):
				train_set = pd.DataFrame()
				test_set = pd.DataFrame()
				for idx_range in idx[:splite_idx]:  # test
					test_set = test_set.append(
					    array.iloc[idx_range[0]:idx_range[1], :], ignore_index=True)
				for idx_range in idx[splite_idx:]:  # train
					train_set = train_set.append(
					    array.iloc[idx_range[0]:idx_range[1], :], ignore_index=True)
				result.append(train_set)
				result.append(test_set)

			# TODO: append method of numpy array
			if isinstance(array, np.ndarray):
				train_set, test_set = [], []
				for idx_range in idx[:splite_idx]:
					test_set.append(array[idx_range[0]:idx_range[1], ])
				for idx_range in idx[splite_idx:]:
					train_set.append(array[idx_range[0]: idx_range[1], ])
				result.append(train_set)
				result.append(test_set)

		return tuple(result)


    def get_sample(self, queries, query, ranking, sampled_queries, qids):
		'''
			Get context base on (qid, query, ranking) and return as list
			Args:
				@param queries: dataFrame stored predict_data
				@param query: query part of target context
				@param ranking: ranking in query group
				@param sampled_queries: 
				@param pids: pid vcorresponding the sampled_queries

			Return:
				Sampled line's token list
		'''
        link_index = queries[(queries.loc[:, 'query'] == query) & (
            queries.ranking == ranking)].reset_index().at[0, 'link_index'][1:-1]
        qid = qids[sampled_queries.index(query)]
        part = link_index.split(',')[0] # name of part_xxxx
        row = int(link_index.split(',')[1]) # line num in file

        if qid[0:2] == 'en':
            with open('data/en_list_result/part-%s' % part.zfill(5), 'r') as fp:
                line = fp.readlines()[row]
        elif qid[0:2] == 'tr':
            with open('data/tr_list_result/part-%s' % part.zfill(5), 'r') as fp:
                line = fp.readlines()[row]
        else:
            print("the name of qid Error\n")
            line = "Error"

		parts = line.split('\x01')
		title = parts[1] * 20  # title has greater weight
        content = parts[2]
        url = parts[0]
        sample = title + content + url
        return sample

	
    def neg_samples(self, queries, n_tasks):
        """
        随机负采样多个样本
		Args:
        	@param queries: (pd.DataFrame) all of the queries data
        	@param n_tasks: set manualy 

        return: [], [[]]
        """
        # adding
        sampled_content = []
        qids = random.sample(set(queries.qid), n_tasks)
        sampled_queries = [queries[(queries.qid == qid) & (queries.ranking == 0)
				].reset_index().at[0, 'query'] for qid in qids]

        for query in sampled_queries:
            # positive sample
            pos_sample = self.get_sample(
                queries, query, 0, sampled_queries, qids)
            # negative sample
            neg_sample = self.get_sample(
                queries, query, 10, sampled_queries, qids)

            sampled_content.append([pos_sample, neg_sample])
        assert len(sampled_queries) == len(sampled_content)
        return sampled_queries, sampled_content



In [ ]:
data = NormalizedData({'data_path': DATA_PATH})
data.normalize_data(doc_info=True, predict_file=True, train_dir=True)


In [ ]:
def my_iter(objs):
	for obj in objs:
		cnt = 0
		for item in obj:
			if cnt >= len(obj):
				break
			yield item
			cnt += 1


In [ ]:
for i in my_iter([['1', '2'], ['a', 'b']]):
	print(i)

In [44]:
x = np.arange(27).reshape((3, 3, 3))
print(x[0:2, ])
print(isinstance(x, np.ndarray))

[[[ 0  1  2]
  [ 3  4  5]
  [ 6  7  8]]

 [[ 9 10 11]
  [12 13 14]
  [15 16 17]]]
True


In [2]:
'a%05d'%1

'a00001'

In [26]:
x = pd.DataFrame()
x['a'] = [1, 2, 3]
x['rank'] = [10, 50, 100]

y = pd.DataFrame()
y['a'] = [10]
y['rank'] = [1000]

In [14]:
x[x['rank'] == 10]['a']

0    1
Name: a, dtype: int64

In [15]:
x.sample(n=2)

,a,rank
0,1,10
2,3,100


In [25]:
for index, i in x.iterrows():
	print(i, type(i))

a        1
rank    10
Name: 0, dtype: int64 <class 'pandas.core.series.Series'>
a        2
rank    50
Name: 1, dtype: int64 <class 'pandas.core.series.Series'>
a         3
rank    100
Name: 2, dtype: int64 <class 'pandas.core.series.Series'>


In [29]:
y.append(x)

,a,rank
0,10,1000
0,1,10
1,2,50
2,3,100


In [30]:
x = eval('(1, 2)')

In [31]:
x[0]

1

In [2]:
ll = [1, 2, 3]
isinstance(ll, list)

True

In [6]:
text = '12312'
not isinstance(text, str)

False

In [12]:
a = None

In [13]:
if a:
	print('1', a)
if a is None:
	print('2', a)

2 None


In [15]:
import math

math.ceil(1 * 0.3)

1

In [13]:
def train_test_split(*arrays, df_col_name=None, test_size=0.33, eval_size=None, shuffle=True, group_size=1):
    """
        Split data to train, eval, test set
        Args:
        @param test_size: float, 
        @param eval_size: default same as test_size
        @param shuffle: determine whether to shuffle
        @group_size: 
        return:
        array[i]-train, array[i]-eval, array[i]-test
    """
    def get_idx(df, target_col):
        res = []
        last_point = 0
        last_key_value = None
        for index, row in df.iterrows():
            if last_key_value is None:
                last_key_value = row[target_col]
            elif last_key_value != row[target_col]:
                res.append((last_point, index))
                last_point = index
                last_key_value = row[target_col]
        return res
    
    result = []
    if eval_size is None:
        eval_size = test_size

    for array in arrays:
        if isinstance(array, pd.DataFrame):
            idx = get_idx(array, df_col_name)
        else:
            idx = [(i, i+group_size) for i in range(0, len(array), group_size)]
        if shuffle:
            random.shuffle(idx)
        splite_idx = math.floor(len(idx) * test_size)
        # for eval set
        train_idx = idx[splite_idx:]
        eval_idx = random.sample(train_idx, math.floor(len(train_idx)*eval_size))
        eval_num = math.ceil(group_size * eval_size) # ! should be enough for sampling

        if isinstance(array, pd.DataFrame):
            train_set = pd.DataFrame()
            test_set = pd.DataFrame()
            eval_set = pd.DataFrame()

            for idx_range in idx[:splite_idx]:
                test_set = test_set.append(
                    array.iloc[idx_range[0]:idx_range[1], :], ignore_index=True)
            for idx_range in idx[splite_idx:]:  # train
                train_set = train_set.append(
                    array.iloc[idx_range[0]:idx_range[1], :], ignore_index=True)
            for idx_range in eval_idx:
                eval_set = eval_set.append(
                    array.iloc[idx_range[0]:idx_range[1], :].sample(n=eval_num), ignore_index=True)

        # TODO: append method of numpy array
        if isinstance(array, np.ndarray):
            train_set, test_set, eval_set = [], [], []
            for idx_range in idx[:splite_idx]:
                test_set.append(array[idx_range[0]:idx_range[1], ])
                
            for idx_range in idx[splite_idx:]:
                train_set.append(array[idx_range[0]: idx_range[1], ])
            for idx_range in eval_idx:
                train_set.append(array[idx_range[0]: idx_range[1], ])
              
           # get eval_set
       


    result.append(train_set)
    result.append(eval_set)
    result.append(test_set)

    return tuple(result)

In [14]:
x = pd.DataFrame()
x['a'] = [i // 10 for i in range(1000)]
x['b'] = ['get%05d'%i for i in range(1000)]

In [15]:
5 // 10

0

In [16]:
xtrain, xeval, x_test = train_test_split(x, df_col_name='a')

In [17]:
print(xtrain)

      a         b
0    89  get00890
1    89  get00891
2    89  get00892
3    89  get00893
4    89  get00894
..   ..       ...
665  18  get00185
666  18  get00186
667  18  get00187
668  18  get00188
669  18  get00189

[670 rows x 2 columns]


In [15]:
# -*- coding: UTF-8 -*-
# from bert import tokenization
import os
import sys
import copy
import json
import random
import math
from itertools import chain
import pandas as pd
import numpy as np
import re
from TurkishStemmer import TurkishStemmer

sys.path.append(os.path.dirname(os.getcwd()))

turkStem = TurkishStemmer()

class TrainData(object):
    def __init__(self, config):
        self.__vocab_path = os.path.join(
            config["bert_model_path"], "vocab.txt")
        self.__output_path = config["output_path"]
        if not os.path.exists(self.__output_path):
            os.makedirs(self.__output_path)
        self._sequence_length = config["sequence_length"]  # 每条输入的序列处理为定长
        self._batch_size = config["batch_size"]

        self.__num_samples = config["num_samples"]

        self.count = 0 # sample num

        self.en_train_data_path = os.path.join(config['data_path'], 'train/train_en.tsv')
        self.tr_train_data_path = os.path.join(config['data_path'], 'train/train_tr.tsv')

        self.predict_path = os.path.join(config['data_path'], 'to_predict.csv')
        self.en_doc_info_path = os.path.join(config['data_path'], 'doc_info/en_list_result/')
        self.tr_doc_info_path = os.path.join(config['data_path'], 'doc_info/tr_list_result/')

        self.part_range = 30000  # num of file num in doc_info 'part-xxxx'


    @staticmethod
    def load_data(file_path):
        """
        :return: DataFrame for all query both in English and Turkish
        """
        data = pd.read_csv(file_path).iloc[:, 1:]
        data = data[data.link_index != '(-1, -1)'].reset_index(drop = True)
        return data


    def sentence_process(self, sentence):
        replacement_pool = [
            ['<br>', ' '],
            ['"', ' '],
            ['\'', ' '],
            ['.', ' '],
            [',', ' '],
            ['?', ' '],
            ['\\', ' '],
            ['`', ' '],
            ['=', ' '],
            ['$', ' '],
            ['/', ' '],
            ['*', ' '],
            [';', ' '],
            ['-', ' '],
            ['^', ' '],
            ['|', ' '],
            ['%', ' '],
            ['\/', ' '],
        ]
        sentence = sentence.lower()
        for rule in replacement_pool:
            sentence = sentence.replace(rule[0], rule[1])
        tokens = sentence.split()
        return tokens


    def getWordsFromURL(self, url, lang):
        words_list = re.compile(r'[\:/?=\-&.,_@%!$0123456789()&*+\[\]]+',re.UNICODE).split(url)
        drop_words = set(['', 'http', 'https', 'www', 'com', '\t', 'm', 'b', 'c', 'd', 'f', 'g', 'h', 'j', 'k', 'l', 'n',
                      'o', 'p', 'q', 'r', 's', 't', 'v', 'w', 'x', 'y', 'z'])

        return [turkStem.stem(word.lower()) for word in words_list if word.lower() not in drop_words]


    def get_tokens(self, series):
        """
            Args:
                @param series: pandas.core.series.Series with lind_index(part_num, line_num)
        """
        link_index = eval(series['link_index'])
        lang = series['qid'][:2]
        if lang == 'en':
            with open(self.en_doc_info_path+'part-%05d'%link_index[0], 'r', encoding='utf8') as fr:
                line = fr.readlines()[link_index[1]]
        elif lang == 'tr':
            with open(self.tr_doc_info_path+'part-%05d'%link_index[0], 'r', encoding='utf8') as fr:
                line = fr.readlines()[link_index[1]]
        else:
            raise NameError('Language type not match')
        
        parts = line.split('\x01')[:3] # ! set as url-title-rank, may need to change
        if len(parts) == 1:
            title_with_content = ''
        elif len(parts) == 2:
            title_with_content = parts[1]
        else:
            title_with_content = (parts[1] + ' ') * 20 + '.' + parts[2]
        # print(len(title_with_content), type(title_with_content))
        res = self.sentence_process(title_with_content) + 20 * self.getWordsFromURL(parts[0], lang)
        return res



    def neg_samples(self, queries, n_tasks):
        """
        随机负采样多个样本
		Args:
        	@param queries: (pd.DataFrame) all of the queries data
        	@param n_tasks: set manually 

        return: [], [[]]
        """
        
        # adding
        # sampled_content = []
        # qids = random.sample(set(queries.qid), n_tasks)
        # sampled_queries = [queries[(queries.qid == qid) & (
        #     queries.ranking == 0)].reset_index().at[0, 'query'] for qid in qids]

        # for query in sampled_queries:
        #     # positive sample
        #     pos_sample = self.get_sample(
        #         queries, query, 0, sampled_queries, qids)
        #     # negative sample

        #     if self.__num_sample > 1:
        #     neg_sample = self.get_sample(
        #         queries, query, 10, sampled_queries, qids)

        #     sampled_content.append([pos_sample, neg_sample])
        # assert len(sampled_queries) == len(sampled_content)
        # return sampled_queries, sampled_content
        
        new_queries = []
        new_sims = []

        qids = random.sample(set(queries.qid), n_tasks)
        cor_qids = list(set(queries.qid) - set(qids))


        for i in range(n_tasks):
            for rank in range(100): 
                if not queries[(queries['qid'] == qids[i]) & (queries['ranking'] == rank)].empty:
                    pos_q = queries[(queries['qid'] == qids[i]) & (queries['ranking'] == rank)] 
            neg_sim = random.sample(cor_qids, self.__num_samples - 1)
            neg_q = queries[queries['qid'].isin(neg_sim)].sample(n=self.__num_samples-1)

            tmp = []
            for index, item in pos_q.append(neg_q).iterrows():
                tmp.append(self.get_tokens(item))

            new_queries.append(pos_q['query'])
            new_sims.append(tmp)
        return new_queries, new_sims

    
    @staticmethod
    def train_test_split(*arrays, df_col_name=None, test_size=0.33, eval_size=None, only_eval=False, shuffle=True, group_size=1):
        """
            Split data to train, eval, test set
            Args:
            @param test_size: float, 
            @param eval_size: default same as test_size
            @param shuffle: determine whether to shuffle
            @group_size: 
            return:
            array[i]-train, array[i]-eval, array[i]-test
        """
        print("=="*20,"\n[train_test_split] Begein split train-eval-test")
        def get_idx(df, target_col):
            res = []
            last_point = 0
            last_key_value = None
            for index, row in df.iterrows():
                if last_key_value is None:
                    last_key_value = row[target_col]
                elif last_key_value != row[target_col]:
                    res.append((last_point, index))
                    last_point = index
                    last_key_value = row[target_col]
                if index % (len(df) // 20) == 0:
                    print("Get index: {}/{}".format(index, len(df)))
            return res
        
        result = []
        if eval_size is None:
            eval_size = test_size
    
        for array in arrays:
            if isinstance(array, pd.DataFrame):
                idx = get_idx(array, df_col_name)
            else:
                idx = [(i, i+group_size) for i in range(0, len(array), group_size)]
            if shuffle:
                random.shuffle(idx)
            # for eval set
            if only_eval:
                split_idx = 0
                train_idx = idx
            else:
                split_idx = math.floor(len(idx) * test_size)
                train_idx = idx[splite_idx:]
            eval_idx = random.sample(train_idx, math.floor(len(train_idx)*eval_size))
            eval_num = math.ceil(group_size * eval_size)
            print("[train_test_split] Finish Initializing")

            if isinstance(array, pd.DataFrame):
                train_set = pd.DataFrame()
                test_set = pd.DataFrame()
                eval_set = pd.DataFrame()

                if split_idx == 0:
                    train_set = copy.copy(array)
                    print("[train_test_split] Finish Train Set(only eval)") 
                else:
                    for idx_range in idx[:split_idx]:
                        test_set = test_set.append(
                            array.iloc[idx_range[0]:idx_range[1], :], ignore_index=True)
                    print("[train_test_split] Get test set")
                    for idx_range in idx[split_idx:]:  # train
                        train_set = train_set.append(
                            array.iloc[idx_range[0]:idx_range[1], :], ignore_index=True)
                    print("[train_test_split] Get train set")
                for idx_range in eval_idx:
                    eval_set = eval_set.append(
                        array.iloc[idx_range[0]:idx_range[1], :].sample(n=eval_num), ignore_index=True)
                print("[train_test_split] Get eval set")

            if isinstance(array, np.ndarray):
                train_set, test_set, eval_set = [], [], []
                
                if split_idx == 0:
                    train_set = copy.copy(array)
                    print("[tarin_test_split] Finish Train Set(Only eval)")
                else:
                    for idx_range in idx[:split_idx]:
                        test_set.append(array[idx_range[0]:idx_range[1], ])
                    print("[train_test_split] Get test set")
                    for idx_range in idx[split_idx:]:
                        train_set.append(array[idx_range[0]: idx_range[1], ])
                    print("[train_test_split] Get train set")
                for idx_range in eval_idx:
                    train_set.append(array[idx_range[0]: idx_range[1], ])
                print("[train_test_split] Get eval set")
                  
        result.append(train_set)
        result.append(eval_set)
        result.append(test_set)

        return tuple(result)



    def trans_to_index(self, texts):
        """
        将输入转化为索引表示
        :param texts: 输入格式：[], 如果is_sim为True，则格式：[[]]
        :return:
        """
        tokenizer = tokenization.FullTokenizer(
            vocab_file=self.__vocab_path, do_lower_case=True)
        input_ids = []
        input_masks = []
        segment_ids = []

        for text in texts:
            if isinstance(text, pd.Series):
                tmp = [x for index, x in text.items()]
                text = ''.join(tmp)
            if isinstance(text, list):
                text = ''.join(text)
            if not isinstance(text, str):
                print(text, type(text))
            text = tokenization.convert_to_unicode(text)
            tokens = tokenizer.tokenize(text)
            tokens = ["[CLS]"] + tokens + ["[SEP]"]
            # print(tokens)
            input_id = tokenizer.convert_tokens_to_ids(tokens)
            input_ids.append(input_id)
            input_masks.append([1] * len(input_id))
            segment_ids.append([0] * len(input_id))
        return input_ids, input_masks, segment_ids


    def padding(self, input_ids, input_masks, segment_ids):
        """
        对序列进行补全
        :param input_ids:
        :param input_masks:
        :param segment_ids:
        :return:
        """
        pad_input_ids, pad_input_masks, pad_segment_ids = [], [], []
        for input_id, input_mask, segment_id in zip(input_ids, input_masks, segment_ids):
            if len(input_id) < self._sequence_length:
                pad_input_ids.append(
                    input_id + [0] * (self._sequence_length - len(input_id)))
                pad_input_masks.append(
                    input_mask + [0] * (self._sequence_length - len(input_mask)))
                pad_segment_ids.append(
                    segment_id + [0] * (self._sequence_length - len(segment_id)))
            else:
                pad_input_ids.append(input_id[:self._sequence_length])
                pad_input_masks.append(input_mask[:self._sequence_length])
                pad_segment_ids.append(segment_id[:self._sequence_length])

        return pad_input_ids, pad_input_masks, pad_segment_ids


    def gen_data(self, file_path):
        """
        生成数据
        :param file_path:
        :return:
        """

        # 1，读取原始数据
        queries = self.load_data(file_path)
        print("read finished")

        return queries


    def gen_task_samples(self, queries, n_tasks):
        """
        生成训练任务和验证任务
        :param queries:
        :param n_tasks:
        :return:
        """
        # 1, 采样
        text_as, text_bs = self.neg_samples(queries, n_tasks)
        self.count += 1
        print("sample {}".format(self.count))

        # 2，输入转索引
        input_ids_a, input_masks_a, segment_ids_a = self.trans_to_index(
            text_as)
        input_ids_a, input_masks_a, segment_ids_a = self.padding(
            input_ids_a, input_masks_a, segment_ids_a)

        input_ids_b, input_masks_b, segment_ids_b = [], [], []
        for text_b in text_bs:
            input_id_b, input_mask_b, segment_id_b = self.trans_to_index(
                text_b)
            input_id_b, input_mask_b, segment_id_b = self.padding(
                input_id_b, input_mask_b, segment_id_b)
            input_ids_b.append(input_id_b)
            input_masks_b.append(input_mask_b)
            segment_ids_b.append(segment_id_b)

        return input_ids_a, input_masks_a, segment_ids_a, input_ids_b, input_masks_b, segment_ids_b


    def next_batch(self, input_ids_a, input_masks_a, segment_ids_a, input_ids_b, input_masks_b, segment_ids_b):
        """
        生成batch数据
        :param input_ids_a:
        :param input_masks_a:
        :param segment_ids_a:
        :param input_ids_b:
        :param input_masks_b:
        :param segment_ids_b:
        :return:
        """
        print("num of epoch: ", len(input_ids_a))
        num_batches = len(input_ids_a) // self._batch_size

        for i in range(num_batches):
            start = i * self._batch_size
            end = start + self._batch_size
            batch_input_ids_a = input_ids_a[start: end]
            batch_input_masks_a = input_masks_a[start: end]
            batch_segment_ids_a = segment_ids_a[start: end]

            batch_input_ids_b = input_ids_b[start: end]
            batch_input_masks_b = input_masks_b[start: end]
            batch_segment_ids_b = segment_ids_b[start: end]

            yield dict(input_ids_a=batch_input_ids_a,
                       input_masks_a=batch_input_masks_a,
                       segment_ids_a=batch_segment_ids_a,
                       input_ids_b=list(chain(*batch_input_ids_b)),
                       input_masks_b=list(chain(*batch_input_masks_b)),
                       segment_ids_b=list(chain(*batch_segment_ids_b)))

In [16]:
config = {
  "model_name": "ltr_pair",
  "epochs": 5,
  "checkpoint_every": 500,
  "eval_every": 500,
  "learning_rate": 2e-5,
  "sequence_length": 32,
  "batch_size": 8,
  "num_samples": 2,
  "train_n_tasks": 10,
  "eval_n_tasks": 500,
  "margin": 0.5,
  "warmup_rate": 0.1,
  "output_path": "output",
  "bert_model_path": "/home/leelin/Project/Pre-trained_models/multi_cased_L-12_H-768_A-12",
  "data": "/home/leelin/Project/DIGIX-2021/digix-2021/data/train_withIndex.csv",
  "ckpt_model_path": "ckpt_model/",
  "data_path": "/home/leelin/Downloads/2021_4_data"
}

In [17]:
TD = TrainData(config)
data = TD.load_data('/home/leelin/Project/DIGIX-2021/digix-2021/data/train_withIndex.csv')



In [18]:
print(type(data))

<class 'pandas.core.frame.DataFrame'>


In [19]:
d_train, d_eval, d_test = TD.train_test_split(data, df_col_name='query', only_eval=True, eval_size=0.33)

[train_test_split] Begein split train-eval-test
Get index: 0/591637
Get index: 29581/591637
Get index: 59162/591637
Get index: 88743/591637
Get index: 118324/591637
Get index: 147905/591637
Get index: 177486/591637
Get index: 207067/591637
Get index: 236648/591637
Get index: 266229/591637
Get index: 295810/591637
Get index: 325391/591637
Get index: 354972/591637
Get index: 384553/591637
Get index: 414134/591637
Get index: 443715/591637
Get index: 473296/591637
Get index: 502877/591637
Get index: 532458/591637
Get index: 562039/591637
Get index: 591620/591637
[train_test_split] Finish Initializing
[train_test_split] Finish Train Set(only eval)
[train_test_split] Get eval set


In [21]:
type(d_train)
print(d_train.shape, d_eval.shape, d_test.shape)

(591637, 5) (1968, 5) (0, 0)


In [22]:
d_train.to_csv('/home/leelin/Desktop/train.csv', index=False)

In [35]:
d_test.to_csv('/home/leelin/Desktop/test.csv', index=False)

In [23]:
d_eval.to_csv('/home/leelin/Desktop/eval.csv', index=False)